# 加法进位实验


<img src="https://github.com/JerrikEph/jerrikeph.github.io/raw/master/Learn2Carry.png" width=650>

In [1]:
import numpy as np
import tensorflow as tf
import collections
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import layers, optimizers, datasets
import os,sys,tqdm

## 数据生成
我们随机在 `start->end`之间采样除整数对`(num1, num2)`，计算结果`num1+num2`作为监督信号。

* 首先将数字转换成数字位列表 `convertNum2Digits`
* 将数字位列表反向
* 将数字位列表填充到同样的长度 `pad2len`


In [2]:
def gen_data_batch(batch_size, start, end):
    '''在(start, end)区间采样生成一个batch的整型的数据
    Args :
        batch_size: batch_size
        start: 开始数值
        end: 结束数值
    '''
    numbers_1 = np.random.randint(start, end, batch_size)
    numbers_2 = np.random.randint(start, end, batch_size)
    results = numbers_1 + numbers_2
    return numbers_1, numbers_2, results

def convertNum2Digits(Num):
    '''将一个整数转换成一个数字位的列表,例如 133412 ==> [1, 3, 3, 4, 1, 2]
    '''
    strNum = str(Num)
    chNums = list(strNum)
    digitNums = [int(o) for o in strNum]
    return digitNums

def convertDigits2Num(Digits):
    '''将数字位列表反向， 例如 [1, 3, 3, 4, 1, 2] ==> [2, 1, 4, 3, 3, 1]
    '''
    digitStrs = [str(o) for o in Digits]
    numStr = ''.join(digitStrs)
    Num = int(numStr)
    return Num

def pad2len(lst, length, pad=0):
    '''将一个列表用`pad`填充到`length`的长度 例如 pad2len([1, 3, 2, 3], 6, pad=0) ==> [1, 3, 2, 3, 0, 0]
    '''
    lst+=[pad]*(length - len(lst))
    return lst

def results_converter(res_lst):
    '''将预测好的数字位列表批量转换成为原始整数
    Args:
        res_lst: shape(b_sz, len(digits))
    '''
    res = [reversed(digits) for digits in res_lst]
    return [convertDigits2Num(digits) for digits in res]

def prepare_batch(Nums1, Nums2, results, maxlen):
    '''准备一个batch的数据，将数值转换成反转的数位列表并且填充到固定长度
    Args:
        Nums1: shape(batch_size,)
        Nums2: shape(batch_size,)
        results: shape(batch_size,)
        maxlen:  type(int)
    Returns:
        Nums1: shape(batch_size, maxlen)
        Nums2: shape(batch_size, maxlen)
        results: shape(batch_size, maxlen)
    '''
    Nums1 = [convertNum2Digits(o) for o in Nums1]
    Nums2 = [convertNum2Digits(o) for o in Nums2]
    results = [convertNum2Digits(o) for o in results]
    
    Nums1 = [list(reversed(o)) for o in Nums1]
    Nums2 = [list(reversed(o)) for o in Nums2]
    results = [list(reversed(o)) for o in results]
    
    Nums1 = [pad2len(o, maxlen) for o in Nums1]
    Nums2 = [pad2len(o, maxlen) for o in Nums2]
    results = [pad2len(o, maxlen) for o in results]
    
    return Nums1, Nums2, results

# 建模过程， 按照图示完成建模

In [3]:
class myRNNModel(keras.Model):
    def __init__(self):
        super(myRNNModel, self).__init__()
        self.embed_layer = tf.keras.layers.Embedding(10, 32)
        self.rnn_layer = tf.keras.layers.SimpleRNN(64, return_sequences=True)  # ✅ SimpleRNN 直接代替 RNNCell
        self.dense = tf.keras.layers.Dense(10)

    def call(self, num1, num2):
        x1 = self.embed_layer(num1)
        x2 = self.embed_layer(num2)
        x = tf.concat([x1, x2], axis=-1)
        x = self.rnn_layer(x)  # ✅ 确保 RNN 层返回完整序列
        logits = self.dense(x)  # ✅ 确保 logits 形状正确
        return logits


In [4]:
@tf.function
def compute_loss(logits, labels):
    losses = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels)
    return tf.reduce_mean(losses)

@tf.function
def train_one_step(model, optimizer, x, y, label):
    with tf.GradientTape() as tape:
        logits = model(x, y)  # ✅ 确保 forward pass 记录了梯度
        loss = compute_loss(logits, label)

    grads = tape.gradient(loss, model.trainable_variables)
    
    if None in grads:  # 检查是否有 None
        print("Gradient computation issue: Some gradients are None")
    
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss


def train(steps, model, optimizer):
    loss = 0.0
    accuracy = 0.0
    for step in range(steps):
        datas = gen_data_batch(batch_size=200, start=0, end=555555555)
        Nums1, Nums2, results = prepare_batch(*datas, maxlen=11)
        loss = train_one_step(model, optimizer, tf.constant(Nums1, dtype=tf.int32), 
                              tf.constant(Nums2, dtype=tf.int32),
                              tf.constant(results, dtype=tf.int64))
        if step%50 == 0:
            print('step', step, ': loss', loss.numpy())

    return loss

def evaluate(model):
    datas = gen_data_batch(batch_size=2000, start=555555555, end=999999999)
    Nums1, Nums2, results = prepare_batch(*datas, maxlen=11)
    logits = model(tf.constant(Nums1, dtype=tf.int32), tf.constant(Nums2, dtype=tf.int32))
    logits = logits.numpy()
    pred = np.argmax(logits, axis=-1)
    res = results_converter(pred)
    for o in list(zip(datas[2], res))[:20]:
        print(o[0], o[1], o[0]==o[1])

    print('accuracy is: %g' % np.mean([o[0]==o[1] for o in zip(datas[2], res)]))


In [5]:
optimizer = optimizers.Adam(0.001)
model = myRNNModel()

In [6]:
train(3000, model, optimizer)
evaluate(model)

step 0 : loss 2.3096125
step 50 : loss 1.922168
step 100 : loss 1.8924037
step 150 : loss 1.8893971
step 200 : loss 1.8801968
step 250 : loss 1.8892667
step 300 : loss 1.8795192
step 350 : loss 1.8812553
step 400 : loss 1.8791264
step 450 : loss 1.8691494
step 500 : loss 1.87731
step 550 : loss 1.869683
step 600 : loss 1.8583075
step 650 : loss 1.8461235
step 700 : loss 1.7964742
step 750 : loss 1.713036
step 800 : loss 1.5763631
step 850 : loss 1.3353884
step 900 : loss 1.0697426
step 950 : loss 0.8923065
step 1000 : loss 0.7619094
step 1050 : loss 0.6524712
step 1100 : loss 0.57445604
step 1150 : loss 0.49940252
step 1200 : loss 0.41953203
step 1250 : loss 0.3545748
step 1300 : loss 0.3036332
step 1350 : loss 0.25437266
step 1400 : loss 0.2074125
step 1450 : loss 0.17823952
step 1500 : loss 0.1438991
step 1550 : loss 0.12187605
step 1600 : loss 0.10489245
step 1650 : loss 0.08956407
step 1700 : loss 0.07700596
step 1750 : loss 0.068475306
step 1800 : loss 0.061394308
step 1850 : loss